# What's the plan?

Well, this is a large data set we have here. So I am going to try a suitable tool called Dask.
Even reading the data will be faster using dask, in comparison to Pandas.
But you probably knew that already. So, I am going one step further, and do some preprocessing using Dask as well!
And by some preprocessing I mean, imputation, scaling and dimension reduction using PCA.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import dask.dataframe as dd
import dask.array as da

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data= dd.read_csv('../input/jane-street-market-prediction/train.csv', blocksize=64e6)
col = data.columns

In [ ]:
data = data[col[7:-1].values]

In [ ]:
from dask_ml.impute import SimpleImputer as si
from dask_ml.preprocessing import StandardScaler as ss
from dask_ml.decomposition import PCA


In [ ]:
imputer = si()
data = imputer.fit_transform(data)

In [ ]:
scaler = ss()
data= scaler.fit_transform(data)

In [ ]:
data = data.to_dask_array(lengths=True)

In [ ]:
import gc
gc.collect()

In [ ]:
data.compute_chunk_sizes()

In next few cells, I am going to first fit a PCA, see what number of top principle components can explain atleast 95% of the variance (named it ***d***).
Then again use a new pca to fit_trarnsform the data, with specific number of components (d).
The reason behinde this syntax is that Dask, unlike Scikit-learn does not accept n_components=0.95 as the input argument. atleast not yet! So I guess it's the way we have to do it right now.

In [ ]:
#     * pass X.to_dask_array(lengths=True)  # for Dask DataFrame (dask >= 0.19)
#     * pass X.compute_chunk_sizes()  # for Dask Array X (dask >= 2.4)
#     * Use a specific SVD solver (e.g., ensure `svd_solver in ['randomized', 'tsqr', 'full']`)
pca = PCA()

In [ ]:
pca.fit(data)

In [ ]:
d = np.argmax(np.cumsum(pca.explained_variance_ratio_)>=0.95) + 1
print(d)

In [ ]:
del(pca)
pca = PCA(n_components = d)
data = pca.fit_transform(data)

In [ ]:
# columns=['Date', 'Weight']
columns=[]
for i in range(1,data.shape[1]+1):
    columns.append('PC_'+str(i))
# columns.append('resp')

In [ ]:
data = data.to_dask_dataframe(columns=columns)

In [ ]:
data.to_csv('./principal_components/pca.csv', single_file=True)

# Final Notes

After saving it, I realized that I have not include the ***weight*** or the ***resp*** in this csv file.
But that's no biggie, I am going to handle it in the next note book which is going to be a deep neural network implemented with with TF2 and keras! 

cheers!